# Extracting Text from PDFs

In this exercise session we will bo going over how to create corpora from PDFs. We'll look at the extraction, the cleaning and some simple annotation. Our goal is to get a simple, verticalized corpus with three rows: Token, Lemma and POS-Tag:
```
This this DET
is be VERB
an an DET
example example NOUN
```



## Introduction

PDFs ([Portable Document File](https://en.wikipedia.org/wiki/PDF)) are a messy file format. They can mix different media types in a very unformalized manner, while employing very unconventional formatting styes.

<img src="img/pdf_p21.png" alt="example" style="width: 500px;"/>

This is a page from the Credit-Suisse Bulletin. In this session, we will be trying to turn this issue into a nice, analyzable format. There are multiple difficulties in this one double-page alone. The title is exloded, some text paths are curved, there are weird symbols in the text.

We'll see what we can do about that. For now, we just need to be aware of the difficulties we are about to encounter.

Another important remark to make about PDFs is the way they store the data that is displayed. PDFs are neither only an image such as a .png-file, nor are they ony composed of encoded characters, such as a .txt-file. If you scan in a page of a book, and disable all OCR ([Optical Character Recognition](https://en.wikipedia.org/wiki/Optical_character_recognition)), you end up with a PDF-file that just displays an image of a book page. For example the letter 'A' is just stored as some pixels that are arranged in a certain way. It's not the representation of the abstract letter 'A', that is encoded in UTF-8 and ASCII by the sequence 1000001.

You can check whether or not the text inside a PDF-document is just an image or actual encoded text by trying to copy-paste it. If you are able to do so, the computer can grab the *abstract* text from the document and put it somewhere else, in a different font and whatnot.

This is important, because our choice of extraction tools depend on it. If it's just the image, we will need an OCR-capable Extractor, such as [tesseract](https://github.com/tesseract-ocr/tesseract) with its Python-wrapper [textract](https://textract.readthedocs.io/en/stable/). But if the text is encoded, we much rather use extraction libraries as [pyPDF2](https://pypi.org/project/PyPDF2/) or what I have used below, [pdfminer3]( https://pdfminersix.readthedocs.io/en/latest/index.html), a fork of [pdfminer.six](https://github.com/pdfminer/pdfminer.six), which is in turn a fork of [pdfminer](https://github.com/euske/pdfminer). PDFs are a strange filetype, and the landscape of libraries to process them appear to match them quite well. You can piece together the informations you need from several different documentations and stackoverflow entries, such as the documentations for [pdfminer.six](https://pdfminersix.readthedocs.io/en/latest/), [pdfminer](https://pdfminer-docs.readthedocs.io/pdfminer_index.html) and the guidance of the Wisepeople of [Stackoverflow](https://stackoverflow.com/questions/26494211/extracting-text-from-a-pdf-file-using-pdfminer-in-python).

## Text Extraction

Using pdfminer is a bit of a hassle, as there are no nice wrapper functions that actually do what we need them to do. 

In [2]:
from pdfminer3.layout import LAParams
from pdfminer3.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer3.converter import PDFPageAggregator
from pdfminer3.pdfpage import PDFPage
from pdfminer3.layout import LTTextBoxHorizontal

In [3]:
document = open('cs-bulletin-01-20-de-web.pdf', 'rb')
rsrcmgr = PDFResourceManager()
#laparams = LAParams()
laparams = LAParams(word_margin=0.4)

device = PDFPageAggregator(rsrcmgr, laparams=laparams)
interpreter = PDFPageInterpreter(rsrcmgr, device)

In [4]:
def textbox_iterator():
    i = 1
    for page in PDFPage.get_pages(document):
        if i != 23:
            i += 1
            continue
        interpreter.process_page(page)
        layout = device.get_result()
        for element in layout:
            if isinstance(element, LTTextBoxHorizontal):
                yield element.get_text()
        i += 1

for textbox in textbox_iterator():
    print(textbox)
    print("----")

« Nascht
heute

----
Milch

----
Gekochtes Gemüse mit Tosa-Sojasauce

----
weniger

----
»

----
Makie Zenimoto (62),  
Ernährungslehrerin an der 
Meiji-Grundschule in Tokio.

----
In keinem Land werden Menschen 
 JAPAN, nirgends 
so alt wie in 
ernähren sie sich so gesund. Dies 
wird schon in der Schule zelebriert. 
Ernährungs lehrerin MAKIE 
ZENIMOTO erklärt, worauf sie 
Wert legt.

----
Interview Sonja Blaschke Fotos Motohiko Hasui

----
S

----
c

----
h

----
w

----
e

----
i

----
n

----
e

----
s

----
c

----
h

----
n

----
i
t
z

----
el a
uf g

----
e
k

----
ochtem Reis mit Misosauce

----
Seit wann achtet man in Japan 
auf ein gesundes Mittagessen 
für Schüler?
Seit etwa 1890 essen japani-
sche Kinder in der Schule zu 
Mittag, früher, um den Armen 
zu helfen. Nach dem Zweiten 
Weltkrieg war man auf Hilfs-
güter der Amerikaner ange-
wiesen – es gab viel Brot. Mit 
der Zeit wurden die Gerichte 
ausgefeilter.

----
Ist das System japanweit 
gleich?
Nein. Mal erstellt die St

## Rough Cleaning

First, I want an iterator over all single lines, and not only over the textboxes. This way, I can then apply the dehyphenation-heuristic below.

In [6]:
def single_lines():
    
    for text_bit in textbox_iterator():
        lines = text_bit.split("\n")
        for line in lines:
            yield line.strip()

for line in single_lines():
    print(line)

« Nascht
heute

Milch

Gekochtes Gemüse mit Tosa-Sojasauce

weniger

»

Makie Zenimoto (62),
Ernährungslehrerin an der
Meiji-Grundschule in Tokio.

In keinem Land werden Menschen
JAPAN, nirgends
so alt wie in
ernähren sie sich so gesund. Dies
wird schon in der Schule zelebriert.
Ernährungs lehrerin MAKIE
ZENIMOTO erklärt, worauf sie
Wert legt.

Interview Sonja Blaschke Fotos Motohiko Hasui

S

c

h

w

e

i

n

e

s

c

h

n

i
t
z

el a
uf g

e
k

ochtem Reis mit Misosauce

Seit wann achtet man in Japan
auf ein gesundes Mittagessen
für Schüler?
Seit etwa 1890 essen japani-
sche Kinder in der Schule zu
Mittag, früher, um den Armen
zu helfen. Nach dem Zweiten
Weltkrieg war man auf Hilfs-
güter der Amerikaner ange-
wiesen – es gab viel Brot. Mit
der Zeit wurden die Gerichte
ausgefeilter.

Ist das System japanweit
gleich?
Nein. Mal erstellt die Stadt-
verwaltung das Menü zentral,
mal überlässt sie das den
Schulen, wie hier im Stadtteil
Koto in Tokio. Mal gibt es
eine Kantine, mal essen Ki

### Dehyphenation

For dehyphenation, I apply this simple heuristic. Basically, two lines get combined without a space-character if the first line ends with a `-`. If the second line starts with a lowercase-letter, the `-` gets deleted as well, if it starts with an UPPERCASE-Letter, the hyphen is kept.
- 1. In the first part of the heuristic, it adds the lines that are kept in the buffer to the line it is currently looking at.
- 2. In the second part, it checks whether the current line ends with a hyhen or not. If it does, we store it in the buffer. If it doesn't, we add it the final text-string and clean the buffer.

In [7]:
text = ""
buffer = ""
last_length = None

for line in single_lines():
    
    # First Part
    if line == "":
        continue
    if buffer != "":
        if line[0].isupper():
            line = buffer + line
        else:
            line = buffer[:-1] + line

            
    # Second Part
    if line.endswith("-"):
        buffer = line
    else:
        text += " " + line
        buffer = ""
        
print(text)

 « Nascht heute Milch Gekochtes Gemüse mit Tosa-Sojasauce weniger » Makie Zenimoto (62), Ernährungslehrerin an der Meiji-Grundschule in Tokio. In keinem Land werden Menschen JAPAN, nirgends so alt wie in ernähren sie sich so gesund. Dies wird schon in der Schule zelebriert. Ernährungs lehrerin MAKIE ZENIMOTO erklärt, worauf sie Wert legt. Interview Sonja Blaschke Fotos Motohiko Hasui S c h w e i n e s c h n i t z el a uf g e k ochtem Reis mit Misosauce Seit wann achtet man in Japan auf ein gesundes Mittagessen für Schüler? Seit etwa 1890 essen japanische Kinder in der Schule zu Mittag, früher, um den Armen zu helfen. Nach dem Zweiten Weltkrieg war man auf Hilfsgüter der Amerikaner angewiesen – es gab viel Brot. Mit der Zeit wurden die Gerichte ausgefeilter. Ist das System japanweit gleich? Nein. Mal erstellt die Stadtverwaltung das Menü zentral, mal überlässt sie das den Schulen, wie hier im Stadtteil Koto in Tokio. Mal gibt es eine Kantine, mal essen Kinder im Klassenzimmer, wie bei u

# Further Cleanup

Some pages aren't as clean as the one we've worked on so far. Especially if they use some weird formatting styles, the layout-parser of pdfminer thinks that different characters of the same word belong to different textboxes. This leaves us with `w o r d s written l i k e t h i s`. The following heuristic takes care of some of these cases.

In addition, I truncate all remaining whitespaces that follow each other to one space-character, and I exchange these `«»` with `'`, because they create problems further down the line.

In [8]:
import re

def get_together(text):
    togethered_text = ""
    buffer = ""
    
    for chunk in text.split(" "):
        if chunk == "":
            continue
            
        if len(chunk) == 1:
            buffer += chunk
            
        else:
            if buffer != "":
                togethered_text += " " + buffer
                buffer = ""
            togethered_text += " " + chunk
            
    return togethered_text

text = get_together(text)
  
text = re.sub(r"\s+", r" ", text)

text = re.sub(r"[«»]",r"'", text)

print(text)

 ' Nascht heute Milch Gekochtes Gemüse mit Tosa-Sojasauce weniger ' Makie Zenimoto (62), Ernährungslehrerin an der Meiji-Grundschule in Tokio. In keinem Land werden Menschen JAPAN, nirgends so alt wie in ernähren sie sich so gesund. Dies wird schon in der Schule zelebriert. Ernährungs lehrerin MAKIE ZENIMOTO erklärt, worauf sie Wert legt. Interview Sonja Blaschke Fotos Motohiko Hasui Schweineschnitz el a uf gek ochtem Reis mit Misosauce Seit wann achtet man in Japan auf ein gesundes Mittagessen für Schüler? Seit etwa 1890 essen japanische Kinder in der Schule zu Mittag, früher, um den Armen zu helfen. Nach dem Zweiten Weltkrieg war man auf Hilfsgüter der Amerikaner angewiesen – es gab viel Brot. Mit der Zeit wurden die Gerichte ausgefeilter. Ist das System japanweit gleich? Nein. Mal erstellt die Stadtverwaltung das Menü zentral, mal überlässt sie das den Schulen, wie hier im Stadtteil Koto in Tokio. Mal gibt es eine Kantine, mal essen Kinder im Klassenzimmer, wie bei uns. Was verstehe

# Text Processing

To process the text, I will be using [spacy](https://spacy.io/). You can use whatever processing-library you like, maybe nltk or a wrapper for the treetagger. I have found spacy to be powerful, easy to implement and very well documented.

I use a special way of creating the doc-object, the documentation can be found [here](https://spacy.io/api/language#pipe). By doing it this way, I can tell spacy NOT to do any syntax-parsing, NER or text-categorisation. It gets a lot faster.

Additionally, I define my own token-type. Most of the pages have a sort of 'watermark' on the bottom line, always the same in one issue. For the first issue of 2020, it's `Bulletin 1/2020`. I've decided that I want this to be a special token in my corpus, as it could by useful to distinguish different pages. I didn't hardcode the year and issue-number, but instead used regex to define the pattern. By doing so, the same code will work for many different issues.

In [9]:
import spacy
import re

nlp = spacy.load("de_core_news_sm")

pattern = r"Bulletin \d+\s*\/\d{4}"

doc = next(nlp.pipe([text], disable=["parser","ner","textcat"]))
indexes = [m.span() for m in re.finditer(pattern, text)]
for start, end in indexes:
    doc.merge(start_idx=start, end_idx=end)
    
for token in doc:
    print(f"{token.text}\t{token.lemma_}\t{token.pos_}")

 	 	SPACE
'	'	PUNCT
Nascht	Nascht	NOUN
heute	heute	ADV
Milch	Milch	NOUN
Gekochtes	Gekochtes	NOUN
Gemüse	Gemüse	NOUN
mit	mit	ADP
Tosa-Sojasauce	Tosa-Sojasauce	NOUN
weniger	wenig	ADV
'	'	PUNCT
Makie	Makie	PROPN
Zenimoto	Zenimoto	PROPN
(	(	PUNCT
62	62	NUM
)	)	PUNCT
,	,	PUNCT
Ernährungslehrerin	Ernährungslehrerin	NOUN
an	an	ADP
der	der	DET
Meiji-Grundschule	Meiji-Grundschule	NOUN
in	in	ADP
Tokio	Tokio	PROPN
.	.	PUNCT
In	In	ADP
keinem	kein	DET
Land	Land	NOUN
werden	werden	AUX
Menschen	Mensch	NOUN
JAPAN	JAPAN	PROPN
,	,	PUNCT
nirgends	nirgends	ADV
so	so	ADV
alt	alt	ADV
wie	wie	ADP
in	in	ADP
ernähren	ernähren	VERB
sie	ich	PRON
sich	sich	PRON
so	so	ADV
gesund	gesund	ADV
.	.	PUNCT
Dies	Dies	PRON
wird	werden	AUX
schon	schon	ADV
in	in	ADP
der	der	DET
Schule	Schule	NOUN
zelebriert	zelebriert	VERB
.	.	PUNCT
Ernährungs	Ernährungs	PROPN
lehrerin	lehrerin	NOUN
MAKIE	MAKIE	PROPN
ZENIMOTO	ZENIMOTO	PROPN
erklärt	erklären	VERB
,	,	PUNCT
worauf	worauf	SCONJ
sie	ich	PRON
Wert	Wert	NOUN
legt	legen	VERB
.	.	PU

# Building the Corpus

With that, we have all the code we need to create our little corpus. I've pieced the single pieces together in the additional script `bulletin_extraction_pipeline.py`. The output of this script is a whole issue of the cs-bulletin in a verticalized format.

I have appended a file wih different bash-commands to perform some basic analysis of the corpus.